In [32]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [33]:
import spacy
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

text = "Hello, how are you?"
doc = nlp(text)

for token in doc:
    print(token.text, token.pos_)


# using spacy to load the english model and tokenizing the text

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Hello INTJ
, PUNCT
how SCONJ
are AUX
you PRON
? PUNCT


In [34]:

pd.set_option('display.max_colwidth',10)

try:
    df=pd.read_csv('/home/one/sAT/IMDB Dataset.csv')
except:
    print("Error in reading the file")

print('first 5 rows of data')
display(df.head())
print(df.info())

# Creating a dataframe in the panda  and doing some basic operations on that

first 5 rows of data


,review,sentiment
0,One of...,positive
1,A wond...,positive
2,I thou...,positive
3,Basica...,negative
4,Petter...,positive


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
None


In [35]:
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
stop

# this block will give us the predefined stopwords in english

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [36]:
import re
def process(text):
    text=re.sub(r'<.#?>',' ', text)
    text=text.lower()
    text=re.sub(r'[^a-z\s]','',text)
    tokens=word_tokenize(text)
    tokens=[word for word in tokens if word not in stop]
    text=' '.join(tokens)
    doc=nlp(text)
    lemitize=[token.lemma_ for token in doc]
    return ' '.join(lemitize)


In [ ]:
print('processing some lines')
df=df.sample(500).copy()
df['cleaned_review']=df['review'].apply(process)
print("cleaned vs origional")
print(df[['review','cleaned_review']].head())

In [38]:
from sklearn.model_selection import train_test_split


df['sentiments']=df['sentiment'].map({'positive':1,'negative':0})

x=df['cleaned_review']
y=df['sentiments']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)

print(f"Training set size: {len(x_train)}")
print(f"Test set size: {len(x_test)}")


Training set size: 400
Test set size: 100


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(max_features=5000)

tfidf_x_train=tfidf.fit_transform(x_train)
tfidf_x_test=tfidf.transform(x_test)



In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear')
model.fit(tfidf_x_train , y_train)

print("Model training complete!")